In [1]:
import numpy as np
import pandas as pd

from compoundDB import inputtools as it
from UpdateDB import Checkpoint as cp
from UpdateDB import CR
from UpdateDB import Update_CII as updater


*** CompoundDB module found. Will check the synonyms table to resolve CAS. ***



#### Load list into pandas

In [2]:
example_df = pd.read_csv('example.csv', sep=',')

#### Checkpoint from CII release version
I make a checkpoint from the stable version just in case I need to roll back the new additions in my database

In [3]:
release_checkpoint = cp.Checkpoint('dummy host','dummy db name','dummy user name','dummy password')

In [4]:
tables_checkpoint = release_checkpoint.get_max_id_for_each_table()

#### Processing example dataframe
First we get CAS and then we generate a list with unique CAS. Finally we'll get the annotations for each CAS from CR database.
Is important to remember that each input should be curated manually to end up having the same format obtained at the end with example_annotations dataframe.

In [5]:
example_df.loc[:,'N.o CAS'] = example_df['N.o CAS'].str.split('/')
example_df.loc[:,'N.o CAS'] = example_df['N.o CAS'].astype(str).apply(lambda x: x.strip('[').strip(']').replace("'","").strip().replace(' and several other',''))

In [6]:
example_copy = example_df.copy()

In [7]:
new_data = {'Sustancia':[],'Listado individual':[], 'N.o CAS':[]}
for i, row in example_df.iterrows():
    subs_row = row['Sustancia']
    pref_name = row['Listado individual']
    cas_row = row['N.o CAS']
    if ',' in cas_row:
        cas_list = cas_row.split(',')
        for cas in cas_list:
            new_data['Sustancia'].append(subs_row)
            new_data['Listado individual'].append(pref_name)
            new_data['N.o CAS'].append(cas.strip())
        example_copy.drop(i, axis=0, inplace=True)
new_df = pd.DataFrame(new_data)

In [8]:
example_conc = pd.concat([example_copy, new_df])
example_conc.reset_index(inplace=True)
example_conc.drop(labels='index',axis=1, inplace=True)
example_conc.rename(columns={'N.o CAS':'CAS'},inplace=True)

#### Connection to CR
I connect to CR and extract annotations for each CAS

In [9]:
ann_df = CR.CR().get_annotations_per_CAS()

#### Preparing dataframes with annotations
Here I create a new dataframe with CR annotations for each CAS in LSR. If I use pd.merge with how='inner' it allows me to recover all CAS from LSR dataframe avoiding empty substances

In [10]:
example_annotations = example_conc.merge(ann_df, on='CAS', how='inner')

In [11]:
example_annotations.sample(10)

,Sustancia,Listado individual,CAS,source_name,original_annotation,annotation,general,category
173,NaN,2-Amino-4-nitrotoluene,99-55-8,REACH Registration,Acute Tox. 3,Acute Tox. 3,Acute Toxicity,Other
119,NaN,2-Naphthylamine,91-59-8,REACH Annex III,Suspected hazardous to the aquatic environment,Suspected hazardous to the aquatic environment,,Aquatic
418,NaN,"4,4’-Oxydianiline",101-80-4,CLP Notification,Acute Tox. 3,Acute Tox. 3,Acute Toxicity,Other
434,NaN,"4,4’-Oxydianiline",101-80-4,REACH Annex VI,PBT,PBT,PBT,PBT
420,NaN,"4,4’-Oxydianiline",101-80-4,CLP Notification,Aquatic Chronic 1,Aquatic Chronic 1,Aquatic Chronic,Aquatic
264,NaN,"4,4’-Diaminodiphenylmethane",101-77-9,REACH Registration,Aquatic Acute 1,Aquatic Acute 1,Aquatic Acute,Aquatic
96,NaN,4-Chloro-o-toluidine,95-69-2,REACH Annex VI,Aquatic Chronic 1,Aquatic Chronic 1,Aquatic Chronic,Aquatic
273,NaN,"4,4’-Diaminodiphenylmethane",101-77-9,SVHC,Carcinogenic (Article 57a),Carcinogenic,Carcinogen,CMR
321,NaN,"3,3’-Dimetoxybenzidine",119-90-4,REACH Annex III,Suspected mutagen,Mutagenic,Mutagen,CMR
324,NaN,"3,3’-Dimetoxybenzidine",119-90-4,REACH Annex III,Suspected persistent in the environment,Suspected persistent in the environment,Persistent,PBT


#### Add dataframe to devel CII
Add new substances to CII or update the ones that are already there. I use original_annotation because it includes EUH annotations, and annotation column doesn't.

In [12]:
updater = updater.UpdateDB(host='dummy host', dbname='dummy db name', user='dummy user', password='dummy password')

Add all the information from the dataframe: substance, chemical identifiers, structure, sources, annotations and how they're related (regulations table)

In [13]:
updater.add_all_information_from_dataframe(dataframe=example_annotations,
                                    class_name_field = 'Sustancia',
                                    preferred_name_field = 'Listado individual',
                                    chem_id_field='CAS',
                                    chem_id_type='casr_number',
                                    sourceName_field='source_name',
                                    regulation_field='original_annotation')

Add only substances by name

In [14]:
updater.add_substances_from_dataframe(dataframe=example_annotations,
                                    class_name_field = 'Sustancia',
                                    preferred_name_field = 'Listado individual')

Add chemical identifiers and substances by name

In [15]:
updater.add_chemical_identifier_from_dataframe(dataframe=example_annotations,
                                            class_name_field = 'Sustancia',
                                            preferred_name_field = 'Listado individual',
                                            chem_id_field='CAS',
                                            chem_id_type='casr_number')

Add only structures (SMILES) if present in the dataframe. If not, it generates the structure from the CAS and stores it

In [16]:
small_strucs = updater.get_substances_with_structure()[:10]

In [17]:
updater.add_structure_from_dataframe(dataframe=small_strucs,
                                    class_name_field = 'class_name',
                                    preferred_name_field = 'preferred_name',
                                    chem_id_field='name',
                                    chem_id_type='casr_number',
                                    smiles_field='structure')

Add only the sources present in the dataframe

In [18]:
updater.add_source_from_dataframe(dataframe=example_annotations,
                                 sourceName_field='source_name')

Add only the annotations (hazards etc...) present in the dataframe

In [19]:
updater.add_annotation_from_dataframe(dataframe=example_annotations,
                                     annotation_field='original_annotation')